In [1]:
import numpy as np
import time
import sys
import random
sys.path.append('./utils')
from Reaction import *
from GenerateStrategies import *

In [2]:
money_mode = np.load('./strategies/money_mode.npy')
supply_mode = np.load('./strategies/supply_mode.npy')
forecast_mode = np.load('./strategies/forecast_mode.npy')

In [3]:
army_infos = Miragine().data

In [4]:
# Decide how to survive.
stage = 0
abyss = False
waits = (160, 270, 370, 580)

In [6]:
'''
Decide which army to choose based on given strategies. Quick and cheap armies are favored.
'''
def decide_move(strategy, opmove, abyss=True):
    if opmove == 6:  # 对付Ninja指定用Samurai
        return 4
    if opmove == 7:  # 对付Novice指定用Ninja
        return 6
    if opmove == 13:  # 对付Mage指定用Heavy SwordMan
        return 8
    if (strategy[1] in (6, 8, 11, 14)) and (strategy[0] not in (6, 8, 9, 11, 12, 14, 15, 16)):
        return strategy[1]
    
    if abyss and army_infos[strategy[1]]['income'] / army_infos[strategy[1]]['money'] > army_infos[strategy[0]]['income'] / army_infos[strategy[0]]['money']:
        return strategy[1]
    else:
        return strategy[0]

In [7]:
t0 = time.time()
last_move = None
op_last_move = None
allow_surprise = False
surprise_area = (1100, 800, 1200, 860)
defense_area = (720, 800, 900, 860)
turn_time = 24.93  # 每一轮的时间
stage = 0
count = 0

'''
Stage 0.
'''
while stage == 0:
    t1 = time.time()

    '''
    Decide my move.
    '''
    try:
        opmove = get_opponent_move()
    except:
        continue

    if opmove == 16 and t1 > 485:  # 一旦对手出High Lord并且危急，我就进入stage 1 应对。
        stage = 1
        break

    my_moves = money_mode[opmove-1]
    # 前几轮没有压倒性优势，不允许奇袭。对方出Lord，我也出High Lord
    surprise = surprise_attack(allow_surprise, stage, opmove, surprise_area)

    if abyss:
        surprise = False

    if surprise != False:
        my_move = surprise
        real_move = my_move
    else:
        if t1 - t0 < waits[0]:
            my_move = 4  # 前几轮刷Samurai赚钱，反正塔有防御，对面攻不过来
            real_move = my_move

        elif t1 - t0 < waits[1]:
            # 然后刷Samurai/Sword Man/Ninja赚钱，反正塔有防御，对面攻不过来
            my_move = [4, 5, 6][count % 3]
            real_move = my_move

        else:
            if t1 - t0 < waits[2]:
                strategy = np.extract(my_moves <= 9, my_moves)
                my_move = decide_move(strategy, opmove, abyss)
            elif t1 - t0 < waits[3]:  # 中间轮不用Lord
                strategy = np.extract(my_moves <= 14, my_moves)
                my_move = decide_move(strategy, opmove, abyss)
                allow_surprise = True
            else:
                strategy = my_moves
                my_move = decide_move(strategy, opmove, abyss)
                allow_surprise = True

            forecast_move = forecast_mode[my_move-1]

            # 预测军种不要跑太快被人家现在这一波给吃掉了，也不要让人家的魔法师堆在一起
            if army_infos[forecast_move]['move_speed'] <= army_infos[my_move]['move_speed'] and opmove not in [7, 10, 13]:
                real_move = my_move if count % 7 else forecast_move
            else:
                real_move = my_move

    click_area(real_move)

    last_move = my_move
    op_last_move = opmove
    count += 1

    time.sleep(0.01)

'''
Stage 1.
'''
t_0 = time.time() - 0.1
turns = 0
op_hist = [16, 0]
my_hist = [16, 0]
surprise_area = (1050, 800, 1200, 860)


while stage == 1:

    allow_surprise = True

    try:
        opmove = get_opponent_move()
    except:
        continue  # 跳过本次循环

    surprise = surprise_attack(allow_surprise, stage, opmove, surprise_area)

    if (my_hist[-1] != 16 and my_hist[-2] != 16) or defense(defense_area):
        my_move = 16
        click_area(my_move)
        time.sleep(6)
        click_area(7)
    
    elif turns % 7 == 1 or turns % 7 == 2:
        my_move = 16
        click_area(my_move)

    elif surprise != False:  # 防御过后，奇袭优先
        my_move = surprise
        click_area(my_move)

    else:
        if opmove == 16:
            my_move = 10  # 没事补点Vampire

        
        #elif my_hist[-1] != 16:
        #    my_move = forecast_mode[my_hist[-1]-1]
        

        else:
            my_moves = supply_mode[opmove-1]
            strategy = np.extract(my_moves <= 14, my_moves)
            for i in strategy:
                if i in (4, 6, 7, 9, 10, 12, 13):  # 找到符合要求的最优军种
                    my_move = i
                    break

        click_area(my_move)

    turns = turns + 1
    op_hist.append(opmove)
    my_hist.append(my_move)

    t_1 = time.time()
    time.sleep(turn_time - (t_1 - t_0) % turn_time)


d:\mypyprojects\miragine\./utils\Reaction.py:103: RuntimeWarning: Mean of empty slice.
  x_mean = np.array(x_all).mean()
e:\veighna_studio\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
d:\mypyprojects\miragine\./utils\Reaction.py:104: RuntimeWarning: Mean of empty slice.
  y_mean = np.array(y_all).mean()


KeyboardInterrupt: 